<a href="https://colab.research.google.com/github/kumaramardeep342/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG - Retrieval-Augmented Generation
RAG is an AI framework for retrieving facts from an external knowledge base to ground large language models (LLMs) on the most accurate, up-to-date information and to give users insight into LLMs' generative process.

## Problem Statement

Document Summarization  of digital documents such as PDFs, Word files, and text files,  and  efficiently extract relevant information for users. Using AI  LLM  models by condensing content into concise, meaningful summaries.

## Loading Files

In [1]:
import os # listing directories , make directory
os.listdir ('/content/drive/MyDrive/Colab Work/Edureka GEN AI + GCP GEN AI Certification/Dataset/Edureka - M9 - RAG Document')

['Recipe-Book.pdf',
 'India Tourism Statistics 2019.pdf',
 'NFDC - Feature Films Catalogue 2024.pdf',
 'Recipe-Book.docx',
 'Food Recipe.txt']

In [2]:
#pdf_file_location = '/content/drive/MyDrive/Colab Work/Edureka GEN AI + GCP GEN AI Certification/Dataset/Edureka - M9 - RAG Document/Recipe-Book.pdf'
# word_file_location = '/content/drive/MyDrive/Colab Work/Edureka GEN AI + GCP GEN AI Certification/Dataset/Edureka - M9 - RAG Document/Recipe-Book.docx'
# text_file_location = '/content/drive/MyDrive/Colab Work/Edureka GEN AI + GCP GEN AI Certification/Dataset/Edureka - M9 - RAG Document/Food Recipe.txt'


## Extract Content

In [3]:
# Step 1: content in the form of pdf,docx,.txt documents , parse the information
# Libraries that can  from these documents and store them.
# These include PDFMiner, PyPDF2, PDFQuery and PyMuPDF.

In [4]:
!pip install PyPDF2 python-docx faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 25.4 MB/s eta 0:00:00


In [5]:
import PyPDF2 , docx # extarct text conent from  pdf files
import re # regular expression
import torch # pytorch
import faiss # facebook database , others - milvus ,pinecone ,chromedb
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [6]:
# reading pdf  file
def extract_text_from_pdf(pdf_file_location):
    with open (pdf_file_location,'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        #print("text_intial",text)
        for page in pdf_reader.pages:
            text += page.extract_text()
       # Remove ".." and newlines
        text = re.sub(r"\.\.+", " ", text)
    return text

# reading word document
def extract_text_from_word(word_file_location):
    word_reader = docx.Document(word_file_location)
    text = ""
    #print("text_intial",text)
    for para in word_reader.paragraphs:
        text += para.text
     # Remove ".." and newlines
    text = re.sub(r"\.\.+", " ", text)
    return text

# reading text document
def extract_text_from_text(text_file_location):
    with open(text_file_location,'r',encoding='latin-1') as text_file:
        text = text_file.read()
    # Remove ".." and newlines
    text = re.sub(r"\.\.+", " ", text)
    return text

In [7]:
# text_from_pdf = extract_text_from_pdf(pdf_file_location)
# text_from_pdf

In [8]:
# text_from_word = extract_text_from_word(word_file_location)
# text_from_word

In [9]:
# text_from_text = extract_text_from_text(text_file_location)
# text_from_text

In [10]:
# text_from_pdf.split()

## Chunking  Content

In [11]:
def  chunk_text(text,chunk_size = 300):
    chunks = []
    words = text.split()
    for i in range(0,len(words),chunk_size):
        chunk = ' '.join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

In [12]:
# chunks = chunk_text(text_from_pdf,chunk_size = 300)
# chunks

## Embeddings

In [13]:
def get_embeddings(chunks):
       inputs = tokenizer(chunks[0], padding=True, truncation=True, return_tensors='pt')
       with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
       return embeddings.numpy()

In [14]:
# embedded_text = get_embeddings(chunks)
# embedded_text

## Automating  -  Extract Content + Chunking Content + Embeddings + Loading in Faiss Vector DB

In [15]:
def extract_chunk_embeddings(directory,chunk_size = 200):
    index = faiss.IndexFlatL2(384) # model output is 384 long vector , in get_embeddings function #outputs.last_hidden_state.mean(dim=1)
    file_chunk_map = []
    for files in os.listdir(directory):
        file_location = os.path.join(directory,files)
        print(f"Creating vector embeddings  for {files}")
        if files.endswith('.pdf'):
            text = extract_text_from_pdf(file_location)
        elif files.endswith('.docx'):
            text = extract_text_from_word(file_location)
        elif files.endswith('.txt'):
            text = extract_text_from_text(file_location)
        else:
            continue
        chunks = chunk_text(text,chunk_size =chunk_size)
        for chunk_id ,chunk in enumerate(chunks):
            embedding = get_embeddings(chunk)
            index.add(embedding) # every chunk of a file is stored in vector database
            file_chunk_map.append((files,chunk_id,chunk))
    return index,file_chunk_map

In [16]:
    # for chunk_id ,chunk in enumerate(chunks):
    #     print(chunk_id,chunk)

In [17]:
directory = '/content/drive/MyDrive/Colab Work/Edureka GEN AI + GCP GEN AI Certification/Dataset/Edureka - M9 - RAG Document'
index,file_chunk_map = extract_chunk_embeddings(directory,chunk_size = 200)
# faiss.write_index(index ,'faiss_index.faiss') # save the index

Creating vector embeddings  for Recipe-Book.pdf
Creating vector embeddings  for India Tourism Statistics 2019.pdf
Creating vector embeddings  for NFDC - Feature Films Catalogue 2024.pdf
Creating vector embeddings  for Recipe-Book.docx
Creating vector embeddings  for Food Recipe.txt


In [18]:
query = " What is BAJRA BISCUIT "
query_embedding = get_embeddings(query)
k = 1
D,I = index.search(query_embedding,k=k) # retrieve top 1 results
results  = []
for i in range(k):
    files,chunk_id,chunk = file_chunk_map[I[0][i]]
    results.append( (files,chunk_id,chunk ))

In [19]:
#query_embedding.shape
#D
#I # this is relvanat information
results

[('Recipe-Book.pdf',
  54,
  'is extracted from mature ripe mangoes and the pulp is pretreated. Mang o pulp is blended with sugar, carrot and permitted class II preservative. Th e homogenized pulp is dehydrated under controlled conditions to obtain fortified mango bar. Th e fortified mango bar is cut and packed in food grade flexible packaging material. Mang o pulp Carrot Sugar F ood acidulant P ermitted class II preservative DEVELOPED BY CSIR-Central Food Technological Research Institute, Mysore15419 gmProtein543 KcalEnergy 37 gmTotal fat SHELF LIFE Eight months in PET bottles and up to 1 year in glass bottles at normal conditions of storage (65% RH and 27ºC). COST Rs. 300 per Kg (subject to inflation) Nutritional Information/100 gramsSESAME BASED Nutritious supplement155Target Group and BenefitsIngredients Th is supplement is formulated for children. Th e product is a high protein and calorie dense product with balanced amino acids and fatty acid. The PDCAAS score of the product is 0

In [20]:
import google.generativeai as genai
from google.colab import userdata

# Get API key and configure Gemini
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


In [21]:
# Initialize the model
model = genai.GenerativeModel('gemini-pro')

def get_gemini_response(query, context):
    # Combine the query and context into a prompt
    prompt = f"""Answer the following question using the provided context.
                Question: {query}
                Context: {context}"""

    try:
        # Generate response
        response = model.generate_content(prompt)

        # Check if response is blocked
        if response.prompt_feedback.block_reason:
            return "Response was blocked due to content safety restrictions."

        return response.text.strip()

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Use the same combined results from your original code
combined_results = " ".join([result[2] for result in results])

# Get response
response = get_gemini_response(query, combined_results)
print(response)

The provided context does not mention anything about BAJRA BISCUIT, so I cannot answer this question from the provided context.
